# Full changesets

## Goals
- Come up with a list of taggers that will make the model an effective flagger

## Workflow
- Download the latest 1000 reviewed changesets from osm full features
- Use the 500 (250 harmful and 250 not harmful) as the training set and the other for testing.
- For every harmful changeset, understand why it was harmful
    - Convert why the changeset was harmful into a feature
- Train a model with the feature
- Predict label for other 500 changesets

## Notes
- Try a lot of taggers - 100 or even a 1000
- What taggers when used by gabbar has the best performance

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
changesets = pd.read_csv('changesets-with-features.csv')
print('Total changesets: {}'.format(changesets.shape[0]))

changesets = changesets.dropna()
print('Changesets after dropna: {}'.format(changesets.shape[0]))

changesets.sample(5)

Total changesets: 949
Changesets after dropna: 865


,changeset_id,harmful,features_created,features_modified,features_deleted,user_id,user_name,user_changesets,user_features,changeset_bbox_area,...,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway
138,47344565,True,2,1,0,5580768,semlohmg,3.0,17.0,2.045847e+03,...,0,0,0,0,0,0,0,0,0,0
372,46946133,True,18,0,0,4891216,Les Pétards,27.0,55.0,3.432960e+13,...,0,0,0,0,0,0,5,0,0,0
630,47470732,False,866,0,0,94578,andygol,22395.0,4677836.0,4.928219e+04,...,0,0,0,0,0,0,0,0,0,0
91,47380463,True,0,1,1,5602193,osmga,3.0,10.0,8.888814e+04,...,0,0,0,0,0,0,0,0,0,0
99,47377981,True,0,1,0,4829742,Anatoly Kozhokar,1.0,1.0,7.468672e+04,...,0,0,0,0,0,0,1,0,0,0


In [4]:
harmful_changesets = changesets[changesets['harmful'] == True]
print('Changesets harmful: {}'.format(harmful_changesets.shape))


not_harmful_changesets = changesets[changesets['harmful'] == False]
print('Changesets harmful: {}'.format(not_harmful_changesets.shape))

Changesets harmful: (423, 39)
Changesets harmful: (442, 39)


In [5]:
features = changesets.columns.values
non_numerical_features = np.array(['changeset_id', 'harmful', 'user_id', 'user_name'])
features = np.setdiff1d(features, non_numerical_features)
print(features)
changesets[features].head()

['aerialway' 'aeroway' 'amenity' 'barrier' 'boundary' 'building'
 'changeset_bbox_area' 'craft' 'emergency' 'features_created'
 'features_deleted' 'features_modified' 'geological' 'highway' 'historic'
 'landuse' 'leisure' 'man_made' 'military' 'natural' 'node_count' 'office'
 'place' 'power' 'public_transport' 'railway' 'relation_count' 'route'
 'shop' 'sport' 'tourism' 'user_changesets' 'user_features' 'waterway'
 'way_count']


,aerialway,aeroway,amenity,barrier,boundary,building,changeset_bbox_area,craft,emergency,features_created,...,railway,relation_count,route,shop,sport,tourism,user_changesets,user_features,waterway,way_count
0,0,0,0,0,0,0,0.000000e+00,0,0,0,...,0,0,0,1,0,0,30.0,2056.0,0,0
1,0,0,0,0,0,0,2.562545e+13,0,0,0,...,0,0,0,0,0,1,2.0,2.0,0,0
2,0,0,0,0,0,0,7.298028e+03,0,0,5,...,0,0,0,0,0,0,5.0,17.0,0,1
3,0,0,0,0,0,0,8.657442e+03,0,0,6,...,0,0,0,0,0,0,5.0,17.0,0,1
4,0,0,24,0,0,2,1.917848e+06,0,0,5,...,0,0,0,6,1,5,571.0,3976.0,0,4


In [6]:
from sklearn.cross_validation import train_test_split

# Feature matrix.
X = changesets[features]

# Target array.
y = changesets['harmful']

# Scale features using RobustScaler which is good for outliers.
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler().fit(X)
X = scaler.transform(X)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=42)
print('Training samples: {}'.format(Xtrain.shape[0]))
print('Testing samples: {}'.format(Xtest.shape[0]))

Training samples: 648
Testing samples: 217


/Users/bkowshik/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

model.fit(Xtrain, ytrain);

# Get predictions.
ymodel = model.predict(Xtest)

In [8]:
from sklearn.metrics import classification_report
print(classification_report(ytest, ymodel))

             precision    recall  f1-score   support

      False       0.17      0.01      0.02       114
       True       0.46      0.95      0.62       103

avg / total       0.31      0.46      0.31       217



In [9]:
# Evaluate a score by cross-validation.
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(model, X, y, cv=3)
np.mean(scores)

0.50755318467256183

/Users/bkowshik/anaconda/lib/python3.5/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [10]:
# Learning curve.
from sklearn.learning_curve import learning_curve

fig, ax = plt.subplots(1, 1)

N, train_lc, val_lc = learning_curve(model, X, y, cv=3, train_sizes=np.linspace(0.1, 1.0, 5))

ax.plot(N, np.mean(train_lc, 1), color='blue', label='Training score')
ax.plot(N, np.mean(val_lc, 1), color='red', label='Validation score')
ax.hlines(np.mean([train_lc[-1], val_lc[-1]]), N[0], N[-1], color='gray', linestyle='dashed')

ax.set_ylim(0, 1)
ax.set_xlim(N[0], N[-1])
ax.set_xlabel('Training size')
ax.set_ylabel('Score')
ax.set_title('Learning curve')
ax.legend(loc='lower right')

plt.tight_layout()

In [11]:
# Add the predictions as a new column.
changesets['prediction'] = model.predict(scaler.transform(changesets[features]))

In [12]:
from sklearn.metrics import confusion_matrix
np.ravel(confusion_matrix(changesets['harmful'], changesets['prediction']))

array([  6, 436,  16, 407])

In [13]:
print('Predicted true: {}'.format(changesets[changesets['prediction'] == True].shape[0]))
print('Predicted false: {}'.format(changesets[changesets['prediction'] == False].shape[0]))

Predicted true: 843
Predicted false: 22


In [14]:
booleans = [True, False]
for actual in booleans:
    for prediction in booleans:
        temp = changesets[(changesets['harmful'] == actual) & (changesets['prediction'] == prediction)]
        print('Actual: {}, Prediction: {}, Changesets: {}'.format(actual, prediction, temp.shape[0]))

Actual: True, Prediction: True, Changesets: 407
Actual: True, Prediction: False, Changesets: 16
Actual: False, Prediction: True, Changesets: 436
Actual: False, Prediction: False, Changesets: 6


In [15]:
changesets[(changesets['harmful'] == False) & (changesets['prediction'] == True)].head()

,changeset_id,harmful,features_created,features_modified,features_deleted,user_id,user_name,user_changesets,user_features,changeset_bbox_area,...,place,power,public_transport,railway,route,shop,sport,tourism,waterway,prediction
469,47491144,False,0,1,0,2219338,RichRico,124816.0,3391525.0,2.983367e+04,...,0,0,0,0,0,0,0,0,0,True
470,47489764,False,0,2,0,236361,NoelB,7790.0,2403385.0,4.419678e+05,...,0,0,0,0,0,0,0,0,0,True
471,47489622,False,1,1,0,1658294,ismaell,231.0,822.0,7.354667e+05,...,0,0,0,0,0,2,0,0,0,True
472,47490850,False,291,110,15,2030995,Bonix-Mapper,12598.0,2915985.0,2.282857e+06,...,0,0,0,0,0,0,0,0,0,True
473,47489440,False,26,5,2,1420318,4b696d,3284.0,344102.0,5.057853e+05,...,0,0,0,0,0,0,0,0,0,True
